# List DataSources

These instructions list the active datasources.

## Prepare Go enviroment

Define global imports, types, variables, and functions.

In [ ]:
import (
	"google.golang.org/grpc"
	"google.golang.org/grpc/credentials/insecure"
    "github.com/senzing-garage/sz-sdk-go-grpc/szabstractfactory"
    "github.com/senzing-garage/sz-sdk-go/senzing"    
)

var (
    ctx = context.TODO()
	grpcAddress    = "localhost:8261"
)    

Create a function for testing error conditions.

In [ ]:
func testErr(err error) {
    if err != nil {
        panic(err)
    }
}

Create a function to construct a Senzing AbstractFactory.

In [ ]:
func getSzAbstractFactory() senzing.SzAbstractFactory {
    grpcConnection, err := grpc.NewClient(grpcAddress, grpc.WithTransportCredentials(insecure.NewCredentials()))
    testErr(err)
    return &szabstractfactory.Szabstractfactory{
    	GrpcConnection: grpcConnection,
    }    
}

Create functions for getting Senzing objects from the SzAbstractFactory.

In [ ]:
func getSzConfig(ctx context.Context, szAbstractFactory senzing.SzAbstractFactory) senzing.SzConfig {
    result, err := szAbstractFactory.CreateConfig(ctx)
    testErr(err)
    return result
}

func getSzConfigManager(ctx context.Context, szAbstractFactory senzing.SzAbstractFactory) senzing.SzConfigManager {
    result, err := szAbstractFactory.CreateConfigManager(ctx)
    testErr(err)
    return result
}

Create a function to pretty print JSON.

In [ ]:
func prettyJSON(str string) (string, error) {
    var prettyJSON bytes.Buffer
    if err := json.Indent(&prettyJSON, []byte(str), "", "    "); err != nil {
        return "", err
    }
    return prettyJSON.String(), nil
}


## Find and print the active datasources

Create an [abstract factory] for accessing Senzing via gRPC.

[abstract factory]: https://en.wikipedia.org/wiki/Abstract_factory_pattern

In [ ]:
var szAbstractFactory = getSzAbstractFactory()

Using the abstract factory, create Senzing objects.
**Note:** This implementation is a little awkward, but it is needed to make global variables using the [gonb] Jupyter Kernel.
Normally it would be something like:

```go
szConfig, err := szAbstractFactory.CreateConfig(ctx)
```

[gonb]: https://github.com/janpfeifer/gonb

In [ ]:
var szConfig = getSzConfig(ctx, szAbstractFactory)
var szConfigManager = getSzConfigManager(ctx, szAbstractFactory)

Get the default configuration and pretty print the datasources.

In [ ]:
%%
ctx := context.TODO()

defaultConfigID, err := szConfigManager.GetDefaultConfigID(ctx)
testErr(err)

configDefinition, err := szConfigManager.GetConfig(ctx, defaultConfigID)
testErr(err)

configHandle, err := szConfig.ImportConfig(ctx, configDefinition)
testErr(err)

dataSources, err := szConfig.GetDataSources(ctx, configHandle)
testErr(err)

dataSourcesPretty, err := prettyJSON(dataSources)
testErr(err)

fmt.Println(dataSourcesPretty)